ising1d.c                                                                  
=========                                                                  
Uses Metropolis algorithm to generate thermal ensemble for the          
    one-dimensional ising chain with free boundary conditions               
    in a magnetic field h.                                                  
                                                                            
Storage: The state of the lattice is stored as spins +-1 in elements 1     
    through N of an array of length N+2.  The free boundary conditions      
    are handled by fixing elements 0 and N+1 of the array to be zero.       

In [1]:
/* update_spin (s, env)
** -----------
** Do a metropolis update on a spin *s whose "environment" is env.
** The environment should be set beforehand so that the dependence
** of the total beta*energy on the value of the selected spin is
** given by (selected spin)*env 
*/


void update_spin(int *s, double env) {
  int spin = *s;
  int newspin = ( drand48() < 0.5 ? 1 : -1 );
  double DeltaBetaE = -(newspin-spin)*env;       /* beta*(E(new) - E(old)) */
  if ( DeltaBetaE <= 0 || drand48() < exp(-DeltaBetaE) ) *s = newspin; 
}

SyntaxError: invalid syntax (943595116.py, line 1)

In [ ]:
/* sweep (spin, N, beta, h)
** -----
** Sweep once through all the sites of the lattice spin of length N,
** trying an update at each site with inverse temperature beta and external
** magnetic field parameter h.
*/
void sweep(int *spin, int N, double beta, double h) {
  for(int ns=1; ns<=N; ns++) {
    update_spin(&(spin[ns]), beta*(spin[ns-1] + spin[ns+1]) + h);
  }
}

In [ ]:
/* InitializeAllSpinUp (spin, N)
** =============================
** Initialize all the N spins of the lattice "spin" to be +1.  Also
** initialize the two fake "boundary" spins to zero to implement free
** boundary conditions.
*/

void InitializeAllSpinUp(int *spin, int N) {
  for(int ns=1; ns<=N; ns++) spin[ns] = 1;
  spin[0] = spin[N+1] = 0;             // no interaction w/ endpoints
}

In [ ]:
void run1d(int nspins, int nsweep, double h, double beta) {
  int ntherm=10000;           // initial sweeps to "thermalize" the system
  const int VisualDisplay=1;  // 1 or 0 to turn on/off display of chains 

  int nmag, ntotal;
  int *spin;
  double expected;
    
  printf("Generating a thermal ensemble for chain of N spins\n");
  printf("with free boundary conditions.\n\n");

  // make room for N spins and two fake boundary spins
  spin = new int[nspins+2];
  InitializeAllSpinUp(spin, nspins);  // cold start

  // sweep ntherm times to thermalize system
  for(int i=0; i<ntherm; i++) sweep(spin,nspins,beta,h);
    
  // now sweep through lattice nsweep times
  nmag = ntotal = 0;
  for(int n=0; n<nsweep; n++) {
    sweep(spin,nspins,beta,h);
    for(int ns=1; ns<=nspins; ns++) {
      nmag += spin[ns];
      ntotal++;
      if (VisualDisplay) {
        if(spin[ns]==1) printf("+");
        if(spin[ns]==-1) printf("-");
      }
    }
    if (VisualDisplay) printf("\n");
   }

   printf("Magnetization: <s> = %lf\n", (double)nmag/ntotal);
   expected = exp(beta)*sinh(h) / 
     sqrt( exp(2*beta)*sinh(h)*sinh(h) + exp(-2*beta) );
   printf("Expected result for infinite length chain over long time: %lf\n",
          expected);
}

In [ ]:
// nspins: 70, nsweep: 30, h: 0, beta: 1.5 
run1d(70,30,0,1.5);

In [ ]:
// nspins: 70, nsweep: 30, h: 0, beta: 0.5 
run1d(70,30,0,0.5);

In [ ]:
// nspins: 70, nsweep: 30, h: 0, beta: 0.1 
run1d(70,30,0,0.1);

In [ ]:
// nspins: 70, nsweep: 30, h: 1, beta: 0.5 
run1d(70,30,1,0.5);

In [ ]:
// nspins: 70, nsweep: 30, h: 0.2, beta: 0.5 
run1d(70,30,.2,0.5);

In [ ]:
// nspins: 70, nsweep: 30, h: -0.2, beta: 0.5 
run1d(70,30,-0.2,0.5);